# Process chunks of data by time
time1 = 1979-1992
time2 = 1992-1999
time3 = 2021-2022

In [1]:
import pickle
import xarray as xr
import numpy as np
import pandas as pd

## TIME1

In [3]:
with open('data/currents_time1.pickle', 'rb') as handle:
    currents1 = pickle.load(handle)
with open('data/chl_time1.pickle', 'rb') as handle:
    chl1 = pickle.load(handle)
with open('data/era5_time1.pickle', 'rb') as handle:
    era51 = pickle.load(handle)
with open('data/sla_time1.pickle', 'rb') as handle:
    sla1 = pickle.load(handle)

In [5]:
currents1

<xarray.Dataset>
Dimensions:  (time: 5114, lat: 177, lon: 241)
Coordinates:
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31
Data variables:
    u_curr   (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    v_curr   (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    ug_curr  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    vg_temp  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan

In [6]:
era51

<xarray.Dataset>
Dimensions:    (time: 5114, lat: 177, lon: 241)
Coordinates:
  * lon        (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * lat        (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31
Data variables:
    u_wind     (time, lat, lon) float32 -0.3151 -0.6693 -0.862 ... -3.49 -3.474
    v_wind     (time, lat, lon) float32 1.49 1.331 1.227 ... 5.068 5.143 5.214
    air_temp   (time, lat, lon) float32 283.5 283.7 283.7 ... 299.4 299.4 299.5
    sst        (time, lat, lon) float32 nan nan nan nan ... 301.3 301.3 301.3
    speed      (time, lat, lon) float32 2.057 1.887 1.83 ... 6.224 6.251 6.303
    direction  (time, lat, lon) float32 91.49 103.9 115.4 ... 124.8 123.9 123.4

In [7]:
ds1 = xr.combine_by_coords([era51, currents1, sla1, chl1])

In [9]:
for var in ds1.data_vars:
    if str(ds1[var].dtype) == 'float64':
        ds1[var].values = ds1[var].astype('float32')

In [10]:
for var in ds1.data_vars:
    if 'chunks' in list(ds1[var].encoding.keys()):
        del ds1[var].encoding['chunks']

In [12]:
with open('shared/data/pickles/ds1.pickle', 'wb') as handle:
    pickle.dump(ds1, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
with open('shared/data/pickles/ds1.pickle', 'rb') as handle:
    ds1 = pickle.load(handle)

In [3]:
ds1.rename({'vg_temp': 'vg_curr'})

<xarray.Dataset>
Dimensions:          (time: 5114, lat: 177, lon: 241)
Coordinates:
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31
Data variables: (12/14)
    CHL_uncertainty  (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    CHL              (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    adt              (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    sla              (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    u_wind           (time, lat, lon) float32 -0.3151 -0.6693 ... -3.49 -3.474
    v_wind           (time, lat, lon) float32 1.49 1.331 1.227 ... 5.143 5.214
    ...               ...
    speed            (time, lat, lon) float32 2.057 1.887 1.83 ... 6.251 6.303
    direction        (time, lat, lon) float32 91.49 103.9 115.4 ... 123.9 123.4
    u_curr           (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    v_curr           (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    ug_curr          (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    vg_curr          (time, lat, lon) float32 nan nan nan nan ... nan nan nan

In [6]:
array_chunk_size = {'time': 100, 'lat': ds1.lat.shape[0], 'lon': ds1.lon.shape[0]}
ds1 = ds1.chunk(array_chunk_size)

In [5]:
ds1 = ds1.rename({'vg_temp': 'vg_curr'})

In [7]:
ds1.to_zarr('shared/data/final_original_no_metadata.zarr/', append_dim='time', mode='a')

## TIME2

In [2]:
with open('data/currents_time2.pickle', 'rb') as handle:
    currents2 = pickle.load(handle)

In [4]:
with open('data/chl_time2.pickle', 'rb') as handle:
    chl2 = pickle.load(handle)
with open('data/era5_time2.pickle', 'rb') as handle:
    era52 = pickle.load(handle)
with open('data/sla_time2.pickle', 'rb') as handle:
    sla2 = pickle.load(handle)

In [5]:
def clean(ds):
    ds.attrs.clear()
    for var in ds.variables:
        ds[var].attrs.clear()
    for var in ds.data_vars:
        if str(ds[var].dtype) == 'float64':
            ds[var].values = ds[var].astype('float32')
        if 'chunks' in list(ds[var].encoding.keys()):
            del ds[var].encoding['chunks']
    return ds

In [3]:
currents2 = currents2.sortby('lat', ascending=False)

In [11]:
currents2

<xarray.Dataset>
Dimensions:  (time: 2556, lon: 241, lat: 177)
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 1999-12-31
  * lon      (lon) float64 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * lat      (lat) float64 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
Data variables:
    u        (time, lon, lat) float32 nan nan nan nan ... 0.1439 0.09165 0.03507
    v        (time, lon, lat) float32 nan nan nan nan ... -0.174 -0.16 -0.1255
    ug       (time, lon, lat) float32 nan nan nan nan ... 0.2314 0.1855 0.1314
    vg       (time, lon, lat) float32 nan nan nan nan ... -0.1567 -0.142 -0.1071

In [6]:
currents2 = clean(currents2)
chl2 = clean(chl2)
era52 = clean(era52)
sla2 = clean(sla2)

In [7]:
ds2 = xr.combine_by_coords([chl2, era52, currents2, sla2])

In [17]:
array_chunk_size = {'time': 100, 'lat': ds2.lat.shape[0], 'lon': ds2.lon.shape[0]}
ds2 = ds2.chunk(array_chunk_size)

In [9]:
ds2 = ds2.rename({
    'u': 'u_curr',
    'v': 'v_curr',
    'ug': 'ug_curr',
    'vg': 'vg_curr'
})

In [10]:
for var in ds2.data_vars:
    if str(ds2[var].dtype) == 'float64':
        ds2[var].values = ds2[var].astype('float32')
    if 'chunks' in list(ds2[var].encoding.keys()):
        del ds2[var].encoding['chunks']

In [16]:
ds2  = ds2.sel(lat=slice(25, 5), lon=slice(60, 80))

In [12]:
ds2 = ds2.transpose('time', 'lat', 'lon')

In [20]:
ds2.to_zarr('shared/minh-notebooks/demonstrated data/new-data-sample.zarr/')

In [19]:
ds2

<xarray.Dataset>
Dimensions:          (time: 2556, lat: 81, lon: 81)
Coordinates:
  * lon              (lon) float32 60.0 60.25 60.5 60.75 ... 79.5 79.75 80.0
  * lat              (lat) float32 25.0 24.75 24.5 24.25 ... 5.75 5.5 5.25 5.0
  * time             (time) datetime64[ns] 1993-01-01 1993-01-02 ... 1999-12-31
Data variables: (12/14)
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    ...               ...
    speed            (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 81, 81), meta=np.ndarray>

In [36]:
ds2.to_zarr('shared/data/final_original_no_metadata.zarr/', append_dim='time', mode='a')

## TIME3

In [38]:
with open('data/currents_time3.pickle', 'rb') as handle:
    currents3 = pickle.load(handle)
with open('data/chl_time3.pickle', 'rb') as handle:
    chl3 = pickle.load(handle)
with open('data/era5_time3.pickle', 'rb') as handle:
    era53 = pickle.load(handle)
with open('data/sla_time3.pickle', 'rb') as handle:
    sla3 = pickle.load(handle)

In [39]:
currents3 = clean(currents3)
chl3 = clean(chl3)
era53 = clean(era53)
sla3 = clean(sla3)

In [41]:
currents3 = currents3.sortby('lat', ascending=False)

In [46]:
ds3 = xr.combine_by_coords([chl3, era53, currents3, sla3])

In [48]:
ds3 = ds3.rename({
    'u': 'u_curr',
    'v': 'v_curr',
    'ug': 'ug_curr',
    'vg': 'vg_curr'
})

In [50]:
ds3 = ds3.transpose('time', 'lat', 'lon')

In [52]:
ds3 = ds3.chunk(array_chunk_size)

In [54]:
ds3.to_zarr('shared/data/final_original_no_metadata.zarr/', append_dim='time', mode='a')

## Combines?

In [56]:
final = xr.open_zarr('shared/data/final_original_no_metadata.zarr/')

In [57]:
final = final.sortby('time')

In [61]:
final = final.chunk(array_chunk_size)

In [76]:
final.to_zarr('shared/data/FINALFINALFINAL.zarr/')

In [14]:
ds = xr.open_zarr('shared/data/FINALFINALFINAL.zarr/')

In [15]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/14)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ...               ...
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     OSCAR, ERA5 Reanalysis, Copernicus Climate Ch...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [10]:
s = ds.sel(time='1979').compute()

## Add back metadata

In [13]:
import json
with open('data/metadata.json') as f:
    meta = json.load(f)
    
with open('data/variables.json') as f:
    v = json.load(f)

In [14]:
meta

{'creator_email': 'minhphan@uw.edu',
 'creator_name': 'Minh Phan',
 'creator_type': 'person',
 'date_created': '2023-07-07',
 'geospatial_lat_max': 32.0,
 'geospatial_lat_min': -12.0,
 'geospatial_lat_resolution': 0.25,
 'geospatial_lat_units': 'degrees_north',
 'geospatial_lon_max': 102.0,
 'geospatial_lon_min': 42.0,
 'geospatial_lon_resolution': 0.25,
 'geospatial_lon_units': 'degrees_east',
 'source': 'OSCAR, ERA5 Reanalysis, Copernicus Climate Change Service (C3S), Copernicus Marine Environment Monitoring Service (CMEMS)',
 'summary': 'Daily mean of 0.25 x 0.25 degrees gridded data from multiple climate variables that may influence the patterns of coastal upwelling in the focused area',
 'time_coverage_end': '2020-12-31T23:59:59',
 'time_coverage_start': '2000-01-01T00:00:00',
 'title': 'Climate Data for Coastal Upwelling Machine Learning Project in Indian Ocean'}

In [70]:
meta['time_coverage_end'] = '2022-12-31T23:59:59'
meta['time_coverage_start'] = '1979-01-01T00:00:00'
meta['date_created'] = '2023-07-07'

In [71]:
final.attrs = meta

In [72]:
final

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/14)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ...               ...
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     OSCAR, ERA5 Reanalysis, Copernicus Climate Ch...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [73]:
for var in final.variables:
    final[var].attrs = v[var]

In [74]:
final

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/14)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ...               ...
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     OSCAR, ERA5 Reanalysis, Copernicus Climate Ch...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [75]:
final = final.chunk({'time': 500, 'lat': ds2.lat.shape[0], 'lon': ds2.lon.shape[0]})